# Store Sales - Time Series Forecasting

Use machine learning to predict grocery sales. [source](https://www.kaggle.com/competitions/store-sales-time-series-forecasting/overview/description)

## Objective

In this Kaggle competition, the goal is to 

> build a model that more accurately predicts the unit sales for thousands of items sold at different Favorita stores.

The evaluation metric for this competition is ***Root Mean Squared Logarithmic Error***.

The `RMSLE` is calculated as:

$$\sqrt{ \frac{1}{n} \sum_{i=1}^n \left(\log (1 + \hat{y}_i) - \log (1 + y_i)\right)^2}$$

where:

- $ n $ is the total number of instances,
     
- $\hat{y}$ is the predicted value of the target for instance (i),
   
- $y_i$ is the actual value of the target for instance (i), and,
 
- $log$ is the natural logarithm.

<!-- sklearn: mean_squared_log_error(y_true, y_pred, squared=False) -->

For each id in the test set, you must predict a value for the sales variable. The file should contain a header and have the following format:

    ```
    id,sales
    3000888,0.0
    3000889,0.0
    3000890,0.0
    3000891,0.0
    3000892,0.0
    etc.
    ```


## Data augmentation

Time series dataset is augmented by 2 more datasets

1. Daily city temperature
2. Country population

### Daily city temperature 

File name and path : data/raw/<>.csv 

[source](https://academic.udayton.edu/kissock/http/Weather/default.htm)

[Data Description](https://academic.udayton.edu/kissock/http/Weather/source.htm)

> The data fields in each file posted on this site are: month, day, year, average daily temperature (F).  We use "-99" as a no-data flag when data are not available. 

Missing data:

> Occasionally, problems with weather station metering equipment result in missing average daily temperatures.  We denote missing data using a “–99” flag.  Depending on the level of accuracy needed, the missing temperatures can be approximated as the mean of the daily temperatures recorded immediately before and after the missing data.  For more accuracy, the user may find other temperature resources.  For example, the mean of the daily minimum and maximum temperatures for many US cities dating back to about 1994 can be found http://www.wunderground.com/. 


Average daily temperature 

> Please note that the “average” daily temperatures on our site are calculated as the mean of 24 hourly readings, rather than the mean of the daily minimum and maximum temperatures.  Thus, our 24-hour average daily temperatures will not be exactly the same as the mean of the daily minimum and maximum temperatures.  However, our research has shown that the standard deviation between the two methods of calculating the average is small (about 1F) with no significant bias.  Thus, the error from filling missing temperatures in our data sets with “(Min+Max)/2” averages is small.

## Country population

This dataset is taken from [wikipedia - Provinces of Ecuador](https://en.wikipedia.org/wiki/Provinces_of_Ecuador)

File name and path : data/raw/provinces.csv 

## Libraries for this research notebook

In [98]:
from datetime import date, datetime
from tqdm.auto import tqdm
import polars as pl
import polars.selectors as cs
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px

# to overcome path issue for src
%reload_ext autoreload
%autoreload 2

from pathlib import Path
import sys

# set the path to the current file
current_file_path = Path().resolve()
print(f"current_file_path is {current_file_path}")

# set the path to the data folder
data_folder_path = current_file_path.parent / 'data/raw/'
print(f"data_folder_path is {data_folder_path}")
# set the path to the src folder
src_folder_path = current_file_path.parent / 'src'
print(f"src_folder_path is {src_folder_path}")

# add the src folder to the system path
sys.path.append(str(src_folder_path))

from data_loader import DBDataLoader
from logger import logging

from pycaret.time_series import TSForecastingExperiment

current_file_path is /home/ubuntu/repos/time-series-forecasting/notebooks
data_folder_path is /home/ubuntu/repos/time-series-forecasting/data/raw
src_folder_path is /home/ubuntu/repos/time-series-forecasting/src


In [99]:
pl.show_versions()

--------Version info---------
Polars:      0.18.4
Index type:  UInt32
Platform:    Linux-5.15.90.1-microsoft-standard-WSL2-x86_64-with-glibc2.35
Python:      3.9.16 | packaged by conda-forge | (main, Feb  1 2023, 21:39:03) 
[GCC 11.3.0]

----Optional dependencies----
numpy:       1.25.1
pandas:      1.5.3
pyarrow:     12.0.1
connectorx:  0.3.1
deltalake:   <not installed>
fsspec:      2023.6.0
matplotlib:  3.7.2
xlsx2csv:    <not installed>
xlsxwriter:  <not installed>


## Data Ingestion


### Load `time_series` data

describe	id	date	store_nbr	family	sales	onpromotion
    str	    f64	str	    f64	        str	      f64	        f64

In [100]:
train = pl.scan_csv("../data/raw/train.csv"
                    , try_parse_dates=True
                    , new_columns=['id', 'date', 'store_nbr', 'family', 'sales', 'onpromotion']
                    , dtypes=[pl.UInt32, pl.Date, pl.Categorical, pl.Categorical, pl.Float64, pl.UInt16]
        )
test = pl.scan_csv("../data/raw/test.csv"
                    , try_parse_dates=True
                    , new_columns=['id', 'date', 'store_nbr', 'family', 'onpromotion']
                    , dtypes=[pl.UInt32, pl.Date, pl.Categorical, pl.Categorical, pl.UInt16]
        )
transactions = pl.scan_csv("../data/raw/transactions.csv"
                           , try_parse_dates=True
                           , new_columns=['date', 'store_nbr', 'transactions']
                           , dtypes=[pl.Date, pl.Categorical,  pl.UInt16]
                )
holidays = pl.scan_csv("../data/raw/holidays_events.csv"
                       , try_parse_dates=True
                       , new_columns=['date', 'type', 'locale', 'locale_name', 'description', 'transferred']
                       , dtypes=[pl.Date, pl.Categorical, pl.Categorical, pl.Categorical, pl.Utf8, pl.Boolean]                       
            )
stores = pl.scan_csv("../data/raw/stores.csv"
                       , new_columns=['store_nbr', 'city', 'state', 'type', 'cluster']
                       , dtypes=[pl.Categorical, pl.Categorical, pl.Categorical, pl.Categorical, pl.UInt8]   
            )
oil = pl.scan_csv("../data/raw/oil.csv"
                  , try_parse_dates=True
        )

train.csv min, max dates

|oldest_date_train|newest_date_train|
|-----------------|-----------------|
|       2013-01-01|       2017-08-15|

In [102]:
# get the respective start and end dates for use to filter daily temperature data
start_date = train.collect().min()["date"][0]
end_date = train.collect().max()["date"][0]

In [103]:
start_date, end_date

(datetime.date(2013, 1, 1), datetime.date(2017, 8, 15))

### Load daily temperature data

In [104]:
# text files for daily temperature has no column names, so we declare it
column_names = ['month', 'day', 'year', 'avg_daily_temp_(F)']

In [105]:
# polars cannot handle space delimited files, so we use pandas and convert later
# specify the column names and missing values used during data capture == '-99'
guayql = pd.read_csv("../data/raw/EQGUAYQL.txt", delim_whitespace=True, names=column_names, na_values='-99')
quito = pd.read_csv("../data/raw/EQQUITO.txt", delim_whitespace=True, names=column_names, na_values='-99')
# use city names as keys
frames = pd.concat([guayql, quito])

In [106]:
print(f'{guayql.shape = }')
print(f'{quito.shape = }')
print(f'concatenated : {guayql.shape[0]+quito.shape[0]}') #  == {ecuador.shape[0]}
# frames['date'] = pd.to_datetime(frames[['year', 'month', 'day']])

# convert concatenated frames into polars dataframe
city_temps = pl.from_pandas(frames)

guayql.shape = (9266, 4)
quito.shape = (7290, 4)
concatenated : 16556


In [107]:
# add 'date' column deriving from the component of dates
city_temps = city_temps.with_columns(
    pl.date(
        pl.col('year'), 
        pl.col('month'), 
        pl.col('day')
    )
.alias('date'))

In [108]:
city_temps.head(3)

month,day,year,avg_daily_temp_(F),date
i64,i64,i64,f64,date
1,1,1995,80.5,1995-01-01
1,2,1995,83.5,1995-01-02
1,3,1995,82.0,1995-01-03


In [109]:
# before we aggregate mean for whole city by date, take care of the NULLs first - 926 entries. take the previous date's value for initial imputation
city_temps.select(pl.all().null_count())
city_temps = city_temps.fill_null('backward')

In [110]:
# calculate the mean temp into a new column 'mean_temp'
city_temps = city_temps.groupby("date").agg(
                    [
                         pl.col("avg_daily_temp_(F)")
                         .mean()
                         .alias('mean_temp')
                    ]).sort('date')

In [111]:
filtered_city_temps = city_temps.filter(
    pl.col("date").is_between(start_date, end_date),
)
print(filtered_city_temps.shape)
print(filtered_city_temps.head())


(1688, 2)
shape: (5, 2)
┌────────────┬───────────┐
│ date       ┆ mean_temp │
│ ---        ┆ ---       │
│ date       ┆ f64       │
╞════════════╪═══════════╡
│ 2013-01-01 ┆ 69.0      │
│ 2013-01-02 ┆ 67.9      │
│ 2013-01-03 ┆ 69.55     │
│ 2013-01-04 ┆ 71.1      │
│ 2013-01-05 ┆ 69.5      │
└────────────┴───────────┘


## Data Analysis (per table)


questions I want to answer:

* which store has 
  * the most sales and promotions ?
  * on which day(s) ?
  * for which product(s), which family of product(s) ?
* does the promotion help increase sales compared to stores with no sales?
* does the promotion help for 
  * days leading to holidays ?
  * days with inclement weather ?
* do oil prices have any impact on stores' sales?
* which period is the best time to offer promotions and increase sales
* which periods sees the most sales? is this value affected by
  * weather
  * holidays
  * promotions
* is there seasonality in the data?
* is this time_series stationary?
* 

### daily city temperatures

In [112]:
fig = px.line(filtered_city_temps, x='date', y='mean_temp', title="Mean Daily temps with NULL values imputed with backward fill")
fig.show()

In [113]:
filtered_city_temps.min()["date"][0]
filtered_city_temps.max()["date"][0]

datetime.date(2017, 8, 15)

### comparing dates across all tables

In [206]:
# check for start and end date of these dataframe
date_ranges = []
df_dict = {'train':train, 'test':test, 'holidays':holidays, 'transactions':transactions, 'oil':oil}
# print('filtered_city_temps')
# print(f'start: {filtered_city_temps.min()["date"][0]}')
# print(f'end: {filtered_city_temps.max()["date"][0]}')
date_ranges.append(('filtered_city_temps', filtered_city_temps.min()["date"][0], filtered_city_temps.max()["date"][0]))
print()
for k,v in df_dict.items():
    # print(k)
    # print(f'start: {v.collect().min()["date"][0]}')
    # print(f'end: {v.collect().max()["date"][0]}')
    # print()
    date_ranges.append((k, v.collect().min()["date"][0], v.collect().max()["date"][0]))

_ = pd.DataFrame(date_ranges, columns=['table', 'start', 'end'])

In [202]:
fig = px.timeline(_, x_start="start", x_end="end", y="table", title="Date ranges of each table")
fig.update_yaxes(autorange="reversed") # otherwise tasks are listed from the bottom up
fig.show()

```
 `holidays` table range from '2012-03-02' to '2017-12-26', comparitively
    `train` table range from '2013-01-01' to '2017-08-15'
```
Laying out the dates, we can see that from the chart, it is clear `holidays` table has extra rows.

This might be the reason the `full_df` *view* has extra 53,460 rows, perhaps also the wrong *join* was utilised.

Gonna need to perform a filter on `holidays` table, just like for `city_temps` before we execute a *join*.

### train dataframe

In [207]:
# calculate grand total of all sales from start to end date to compare later
sales_grand_total = train.select(pl.col('sales').sum()).collect()[0].to_series().view()[0]
print(f"Grand total of all sales: ${sales_grand_total:,.2f}")


Grand total of all sales: $1,073,644,952.20


In [208]:
train.collect().describe()

describe,id,date,store_nbr,family,sales,onpromotion
str,f64,str,str,str,f64,f64
"""count""",3.000888e6,"""3000888""","""3000888""","""3000888""",3.000888e6,3.000888e6
"""null_count""",0.0,"""0""","""0""","""0""",0.0,0.0
"""mean""",1500443.5,null,null,null,357.775749,2.60277
"""std""",866281.891642,null,null,null,1101.997721,12.218882
"""min""",0.0,"""2013-01-01""",null,null,0.0,0.0
"""max""",3.000887e6,"""2017-08-15""",null,null,124717.0,741.0
"""median""",1500443.5,null,null,null,11.0,0.0
"""25%""",750222.0,null,null,null,0.0,0.0
"""75%""",2.250666e6,null,null,null,195.848,0.0


train.columns = ['id', 'date', 'store_nbr', 'family', 'sales', 'onpromotion']

In [117]:
# q = train.select(
#     "date",
#     "family",
#     pl.col("date").dt.year,
#     pl.col("sales")
#     .sum()
#     .over(["date", "family"])
#     .alias("sales_total_by_year_family"),
#     pl.col("sales").mean().alias("avg_sales"),
# )
q = train.with_columns(
   pl.sum("sales").over(["date", "family"]).alias("daily_total")
)

print(q)     

train_yearly_sales = q.collect();
train_yearly_sales.tail(5)

naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan)

 WITH_COLUMNS:
 [col("sales").sum().over([col("date"), col("family")]).alias("daily_total")]

    CSV SCAN ../data/raw/train.csv
    PROJECT */6 COLUMNS


id,date,store_nbr,family,sales,onpromotion,daily_total
u32,date,cat,cat,f64,u16,f64
3000883,2017-08-15,"""9""","""POULTRY""",438.133,0,17586.709986
3000884,2017-08-15,"""9""","""PREPARED FOODS…",154.553,1,4641.52298
3000885,2017-08-15,"""9""","""PRODUCE""",2419.729,148,125108.971
3000886,2017-08-15,"""9""","""SCHOOL AND OFF…",121.0,8,2530.0
3000887,2017-08-15,"""9""","""SEAFOOD""",16.0,0,970.177005


In [118]:
train_sales = train.groupby("date").agg(
                    [
                         pl.col("sales")
                         .sum()
                         .alias('sales_total')
                    ]).sort('date')



In [119]:
train_sales.collect().head(3)

# train.sales.shape = (1684, 2)

date,sales_total
date,f64
2013-01-01,2511.618999
2013-01-02,496092.417944
2013-01-03,361461.231124


### by date: daily total sales

In [120]:
fig = px.line(train_sales.collect(), x='date', y='sales_total', title="Total Sales from 2013-01-01 to 2017-08-15")
fig.show()

In [121]:
# train_sales.

[ ] Add annotation for New Year's Day having no sales.
[ ] Want to annotate "big" holidays like CNY / Xmas for example? Means need to learn what is the "BIG" holiday events; is it by length? Like CNY in SG is 2 days; in China is 7 days...

### by store_nbr: sales frequency

for each `sales`, how often is the sale for a specific `store_nbr`?

In [181]:
q = train.groupby('store_nbr').agg(
                    [
                         pl.col('sales')
                         .count()
                         .alias('sales_freq')
                    ]).sort(pl.col('store_nbr').cat.set_ordering("lexical"))

freq_count_by_store_nbr = q.collect()
freq_count_by_store_nbr.head()

store_nbr,sales_freq
cat,u32
"""1""",55572
"""10""",55572
"""11""",55572
"""12""",55572
"""13""",55572


In [182]:
fig = px.bar(freq_count_by_store_nbr
             , x='store_nbr'
             , y='sales_freq'
             , title="Sales Frequency by STORE NUMBER from 2013-01-01 to 2017-08-15"
            #  , orientation='h'
             )
fig.show()

### by str_nbr: total sales

In [124]:
q = train.groupby("store_nbr").agg(
                    [
                         pl.col("sales")
                         .sum()
                         .alias('sales_total')
                    ]).sort('sales_total', descending=True)

total_sales_by_store_nbr = q.collect()
total_sales_by_store_nbr.head()

store_nbr,sales_total
cat,f64
"""44""",6.2088e7
"""45""",5.4498e7
"""47""",5.0948e7
"""3""",5.0482e7
"""49""",4.3420e7


In [131]:
fig = px.bar(total_sales_by_store_nbr
             , x='store_nbr'
             , y='sales_total'
             , title="Total Sales by STORE NUMBER from 2013-01-01 to 2017-08-15, in descending values"
             )
fig.show()

[ ] find out currency symbol or measurememnt for Ecuador

`store_nbr` = 44 is the best perfomer of all the stores, generating sales total of > 60million while `store_nbr` = 52 is the worst performer, generating a little bit over 2.6million. However, comparing just by sales value alone is not a correct metrics since store number 44 might be located in a populous area, while store#52 is not in a populous city.

A more accurate measure needs to be compared to city's population and get the sales mean value per 10,000 people (or whichever is the smallest measure for the smallest city, rounded up to the nearest thousands.)

### by family: sales frequency

for each `sales`, how often is the sale for a specific `family` of products?

In [126]:

q = train.groupby("family").agg(
                    [
                         pl.col("sales")
                         .count()
                         .alias('sales_freq')
                    ]).sort('sales_freq', descending=True)

freq_count_by_family = q.collect()
freq_count_by_family.head()

family,sales_freq
cat,u32
"""CELEBRATION""",90936
"""BOOKS""",90936
"""PRODUCE""",90936
"""GROCERY II""",90936
"""LAWN AND GARDE…",90936


In [127]:
fig = px.bar(freq_count_by_family
             , y='family'
             , x='sales_freq'
             , title="Sales Frequency by FAMILY of products from 2013-01-01 to 2017-08-15"
             , orientation='h')
fig.show()

### by family: total sales

In [132]:
q = train.groupby("family").agg(
                    [
                         pl.col("sales")
                         .sum()
                         .alias('sales_total')
                    ]).sort('sales_total')#, descending=True)

sales_by_family = q.collect()
sales_by_family.head()

family,sales_total
cat,f64
"""BOOKS""",6438.0
"""BABY CARE""",10051.0
"""HOME APPLIANCE…",41601.0
"""HARDWARE""",103470.0
"""MAGAZINES""",266359.0


In [133]:
fig = px.bar(sales_by_family
             , y='family'
             , x='sales_total'
             , title="Total Sales by FAMILY of products from 2013-01-01 to 2017-08-15"
             , orientation='h')
fig.show()

In [ ]:
train.collect()

In [ ]:
CREATE OR REPLACE VIEW full_df AS
    SELECT 
        tr.id,
        tr.family,
        tr.sales,
        tr.onpromotion,
        city,
        state,
        cluster,
        locale,
        locale_name,
        description,
        transferred,
        st.`type`,
        hols.`type` hol_type,
        tr.store_nbr,
        tr.`date`,
        YEAR(tr.`date`) `year`,
        MONTH(tr.`date`) `month`,
        DAY(tr.`date`) `day_of_month`,
        transactions,
        o.dcoilwtico
    FROM
        train AS tr
            LEFT JOIN
        stores AS st ON tr.store_nbr = st.store_nbr
            LEFT JOIN
        holidays_events hols ON tr.`date` = hols.`date`
            LEFT JOIN
        transactions txn ON tr.`date` = txn.`date`
            AND tr.store_nbr = txn.store_nbr
            LEFT JOIN
        oil o ON tr.`date` = o.`date`;


In [ ]:


# profile = ProfileReport(
#     site,
#     tsmode=True,
#     type_schema=type_schema,
#     sortby="Date Local",
#     title="Time-Series EDA for site 3003",
# )

# profile.to_file("report_timeseries.html")

In [ ]:
from ydata_profiling import ProfileReport
profile = ProfileReport(train.collect().to_pandas(), title="ProfileReport train", tsmode=True)
profile.to_file("../artifacts/reports/train_ProfileReport.html")

In [ ]:
equador = city_temperature.select(['Country','State','City', 'Month', 'Day', 'Year', 'AvgTemperature']).filter(pl.col("Country") == "Equador").unique().collect()


In [ ]:
equador

In [ ]:
city_temperature.select(pl.col(["Country"])).unique().collect()

In [ ]:
equador = city_temperature.filter(pl.col("Country") == "Equador").collect()

In [ ]:
equador

~~DF loaded confirm: 1972674 rows × 14 columns~~

DF loaded confirm: 3,054,348 rows × 20 columns

## Data cleaning

In [ ]:
df['date'] = pl.to_datetime(df['date'])

In [ ]:
df.info()

In [ ]:
groupby_store = df.groupby(by=['store_nbr', 'family'], group_keys=True).agg('sum', 'mean')

In [ ]:
groupby_store.info()

## Data profile

In [ ]:
# from ydata_profiling import ProfileReport

In [ ]:
# profile = ProfileReport(df, title="ProfileReport full_df")
# # # profile.to_notebook_iframe()
# profile.to_file("../artifacts/reports/full_df_ProfileReport.html")

## Data preprocessing

## autoML with pycaret

EDA and ML


In [ ]:
# check installed version
import pycaret
pycaret.__version__

In [ ]:
# df.columns = ['id', 'family', 'sales', 'onpromotion', 'city', 'state', 'cluster',
#        'locale', 'locale_name', 'description', 'transferred', 'type',
#        'hol_type', 'store_nbr', 'year', 'month',
#        'day_of_month', 'transactions', 'dcoilwtico']

In [ ]:
#  simplify dataset
data = df.groupby(by=[], group_keys=True).agg('sum', 'mean')

In [ ]:
data.head()

In [ ]:
import matplotlib
%matplotlib inline
# plot the dataset
data.plot();

In [ ]:
# import pycaret time series and init setup
from pycaret.time_series import *
s = setup(data, fh = 3, session_id = 123)  # fh = 3 --> 3 folds

In [ ]:
eda(display="bokeh")

In [ ]:
# check statistical tests on original data
check_stats()

In [ ]:
# compare baseline models
best = compare_models()

In [ ]:
# plot forecast
plot_model(best, plot = 'forecast')

In [ ]:
# plot forecast for 36 months in future
plot_model(best, plot = 'forecast', data_kwargs = {'fh' : 36})

In [ ]:
# residuals plot
plot_model(best, plot = 'residuals')

In [ ]:
# predict on test set
holdout_pred = predict_model(best)

In [ ]:
# show predictions df
holdout_pred.head()

In [ ]:
# generate forecast for 36 period in future
predict_model(best, fh = 36)

In [ ]:
# save pipeline
save_model(best, 'my_first_pipeline')

In [ ]:
# load pipeline
loaded_best_pipeline = load_model('my_first_pipeline')
loaded_best_pipeline